In [ ]:
import sys
import os

current_dir = os.getcwd()

project_root = os.path.abspath(os.path.join(os.path.dirname(current_dir), '.'))
if project_root not in sys.path:
    sys.path.append(project_root)

import pinns

# For cleaner output.
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

Given $x_0$ and $y_0$, find such $x(t)$ and $y(t)$ : $\mathbb{R} \to \mathbb{R}$, so that

$$\frac{\mathrm{d}x}{\mathrm{d}t} = \alpha x - \beta x y$$
$$\frac{\mathrm{d}y}{\mathrm{d}t} = \delta x y - \gamma y$$
$$x(0) = x_0, \quad y(0) = y_0$$

We consider problem with following given parameters and initial values: $\alpha = 0.4, \beta = 0.1, \delta = 0.1, \gamma = 0.6$ and $x_0 = 5, y_0 = 5$ on a domain $[0, 38]$.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

We will solve this problem numerically.

In [ ]:
from scipy.integrate import solve_ivp

# We fix variables for clarity.
T = 38
alpha, beta, delta, gamma = 0.4, 0.1, 0.1, 0.6
x0, y0 = 5.0, 5.0

def lotka_volterra(t, u, alpha, beta, delta, gamma):
            x, y = u
            dx_dt = alpha * x - beta * x * y
            dy_dt = delta * x * y - gamma * y
            return [dx_dt, dy_dt]

t = torch.linspace(0, T, 128)
solution = solve_ivp(lotka_volterra, 
                        (0, T),
                        [x0, y0],
                        args=(alpha, beta, delta, gamma), 
                        t_eval=t)
x = solution.y[0]
y = solution.y[1]

fig = plt.figure(figsize=(5, 3))
plt.plot(t, x, label = 'X(t)')
plt.plot(t, y, label = 'Y(t)')
plt.grid()
plt.legend()
plt.show()

Our neural network for this task will have one input and two outputs.

In [ ]:
from pinns.samplers import ConstantSampler, RandomRectangularSampler

constraints_sampler = ConstantSampler((
    torch.tensor([[0.]]),
    torch.tensor([x0, y0])
))

domain = {'t': [0, T]}
collocation_sampler = RandomRectangularSampler(domain, 2048, return_dict=False)

test_points_sampler = ConstantSampler((
    t.view(-1, 1), 
    torch.tensor([x, y]).T
    ))

In [ ]:
from pinns.derivatives import Derivative

d = Derivative(method = 'autograd')

def loss(
    cstr_pts, cstr_pred, cstr_vals,
    coll_pts, coll_pred,
    alpha = 0.4, beta = 0.1, delta = 0.1, gamma = 0.6
    ):
    
    def init_loss(u0, t0):
        return torch.mean(torch.square(u0 - cstr_vals))

    def ode_loss(u, t):
        x = u[:, [0]]
        y = u[:, [1]]
        
        dX = d(x, t)
        dY = d(y, t)
        
        LdX = torch.mean(torch.square(dX - alpha * x + beta * x * y))
        LdY = torch.mean(torch.square(dY - delta * x * y + gamma * y))
        
        return LdX, LdY
    
    return init_loss(cstr_pred, cstr_pts), *ode_loss(coll_pred, coll_pts)

In [ ]:
from pinns import Trainer
from pinns.models import FF
from pinns.activations import Sin
from pinns.optimizers import Adam
from pinns.errors import rmse

pinn = FF([1] + [64, 64, 64, 64] + [2], activ=nn.Tanh(), biases=True)
pinn.init(nn.init.kaiming_normal_)
print(f'Model has {pinn.count_parameters()} trainable parameters.')

adam = Adam(pinn, lr = 1e-2)

trainer = Trainer(
    loss,
    pinn,
    constraints_sampler,
    collocation_sampler,
    loss_coefs=[0.8, 0.2, 0.2],    # Coefficients are very important.
    test_points_sampler=test_points_sampler
)

num_iters = 25000
save_every = 500

def make_plot():
    if trainer.iter == 0 or trainer.iter % save_every == 0 or trainer.iter == num_iters:
        preds = pinn.predict(test_points_sampler()[0]).detach()
        np.save(f'./.temp/lv_{trainer.iter}.npy', preds.numpy())

trainer.train(
    num_iters=num_iters,
    optimizers=[(0, adam)],
    validate_every=1,
    error_metric = rmse,
    at_training_start_callbacks=[make_plot],
    at_epoch_end_callbacks=[make_plot],
    at_training_end_callbacks=[make_plot]
    )

In [ ]:
trainer.evaluate(rmse)

In [ ]:
# pinn.model = torch.load('./very_good_model_dont_delete.pt')

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].plot(trainer.loss_history, label='Loss')
axs[0].plot(range(0, trainer.iter + 1, 1), trainer.error_history, label='L2')
axs[0].grid()
axs[0].set_yscale('log')
axs[0].legend()

preds = pinn.predict(t.reshape(-1, 1))
axs[1].plot(t, x, label = 'x(t)')
axs[1].plot(t, y, label = 'y(t)')
axs[1].plot(t, preds[:,0].detach(), label='X(t)', linestyle=':')
axs[1].plot(t, preds[:,1].detach(), label='Y(t)', linestyle=':')
axs[1].grid()
axs[1].legend()

plt.show()

In [ ]:
from PIL import Image
import imageio
from joblib import Parallel, delayed

from tqdm.notebook import tqdm_notebook as tqdm

def save_animation(files, path, duration=5, fps=60, loop=0, type='mp4', processors=2, ):
    
    fig = plt.figure(figsize=(5, 3))
    xlim = t.min(), t.max()
    
    def plot(i):
        preds = np.load(files[i])
        # Set plot limits and labels
        plt.xlim(xlim)
        plt.plot(t, x, label = 'x(t)')
        plt.plot(t, y, label = 'y(t)')
        plt.plot(t, preds[:,0], label='X(t)', linestyle=':')
        plt.plot(t, preds[:,1], label='Y(t)', linestyle=':')
        plt.grid()
        plt.legend()
        fig.savefig(f'./.temp/frame_{i}.png', dpi=300)
        fig.clear()
        
    # Number of frames
    num_frames = len(files)

    # Parallelize the plotting function
    Parallel(n_jobs=processors, verbose=4)(delayed(plot)(i) for i in range(num_frames))
    
    if type == 'mp4':
        writer = imageio.get_writer(path, fps=fps)
        for i in range(len(files)):
            writer.append_data(imageio.imread(f'./.temp/frame_{i}.png'))
        writer.close()
        
    if type == 'gif':
        imgs = [Image.open(f'./.temp/frame_{i}.png') for i in range(len(files))]
        imgs[0].save(path, save_all=True, append_images=imgs[1:], duration=duration, fps=fps, loop=loop)
    
files = [f'./.temp/lv_{i}.npy' for i in range(0, trainer.iter, save_every)]
save_animation(files, './.results/lotka-volterra animation.gif', type='gif', processors=8)